In [8]:
# import libraries 

from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib
import datetime
import pandas as pd

In [2]:
# Connect to required website and specify the headers

URL = 'https://www.amazon.ca/Queen-70-Glorious-Years/dp/1909741825/ref=zg_bsnr_933484_2?_encoding=UTF8&psc=1&refRID=FZSB4GWKGTZBP9FZYMEV'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0", "Accept-Encoding":"gzip, deflate, br", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [3]:
#Create elements required for web scraping

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

#Title of the product
title = soup2.find(id='productTitle').get_text()

#Price of the product
price = soup2.find(id='price').get_text()

#Print the title and price
print(title)
print(price)


     The Queen: 70 Glorious Years
    

                          $31.30
                         


In [4]:
# Clean up the data to remove spaces
price = price.strip()[1:]
title = title.strip()

print(title)
print(price)

The Queen: 70 Glorious Years
31.30


In [6]:
#Timestamp
today = datetime.date.today()

print(today)

2022-01-17


In [7]:
# Create CSV and write headers and data into the file

import csv 

header = ['Title', 'Price', 'Date']
data = [title, price, today]

with open('DatasetAmazon.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [10]:
#Display the contents of the csv

df = pd.read_csv(r'DatasetAmazon.csv')

print(df)

                          Title  Price        Date
0  The Queen: 70 Glorious Years   31.3  2022-01-17


In [11]:
#Updating the contents of the csv
with open('DatasetAmazon.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [12]:
#Create a function to perform to check price of a product

def check_price():

    #Setting up the prodcut url for which price needs to be checked
    URL = 'https://www.amazon.ca/Queen-70-Glorious-Years/dp/1909741825/ref=zg_bsnr_933484_2?_encoding=UTF8&psc=1&refRID=FZSB4GWKGTZBP9FZYMEV'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0", "Accept-Encoding":"gzip, deflate, br", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    page = requests.get(URL, headers=headers)

    #Creating soup objects
    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    #Title and price data obtained
    title = soup2.find(id='productTitle').get_text()

    price = soup2.find(id='price').get_text()

    #Clean the data
    price = price.strip()[1:]
    title = title.strip()
    today = datetime.date.today()

    #Specify column names for the csv file
    header = ['Title', 'Price', 'Date']
    data = [title, price, today]

    #Update the contents of the file
    with open('DatasetAmazon.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

In [13]:
# Function to send a mail alert for changes in price

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login('abc@gmail.com','xxxxxxxxxxxxxx')
    
    subject = "The price of the product X is below the required price Y"
    body = f"OP, the price of the product you want has reduced. Please refer the following link to make a decision regarding the purchase:{URL}"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail('abc@gmail.com',msg)

In [ ]:
#Check price regularly

while(True):
    check_price()
    time.sleep(86400)

    if price < 15:
      send_mail()